<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#center-detect" data-toc-modified-id="center-detect-1">center detect</a></span></li><li><span><a href="#hrnet" data-toc-modified-id="hrnet-2">hrnet</a></span><ul class="toc-item"><li><span><a href="#Check-for-incomplete-h5-files-(will-need-to-be-re-run)" data-toc-modified-id="Check-for-incomplete-h5-files-(will-need-to-be-re-run)-2.1">Check for incomplete h5 files (will need to be re-run)</a></span></li></ul></li><li><span><a href="#triangulate" data-toc-modified-id="triangulate-3">triangulate</a></span></li><li><span><a href="#gimbal" data-toc-modified-id="gimbal-4">gimbal</a></span></li></ul></div>

In [1]:
import glob, os
from os.path import join, exists
import numpy as np
import h5py

In [2]:
good_sessions = [
    
    # J016 are older, may want to exclude from ultimate model
    '20230904_J01601',
    '20230905_J01601',
    
    '20230904_J01602',
    '20230905_J01602',
    
    '20230904_J01701',
    '20230905_J01701',
    '20230911_J01701',
    '20230912_J01701',
    
    '20230905_J01702',
    '20230911_J01702',
    '20230912_J01702',
    
    '20230904_J01703',
    '20230908_J01703',
    '20230911_J01703',
    '20230912_J01703',
]

video_glob = "*.avi"
# video_glob = "*.COMPRESSED.mp4"

base_path = '/n/groups/datta/Jonah/20231121_6cam_sniff/kp_detection/'
path_to_centernet = join(base_path, 'apply_centernet.py')
path_to_hrnet = join(base_path, 'apply_hrnet.py')
path_to_triangulate = join(base_path, 'apply_robust_triangulate.py')
# path_to_gimbal = join(base_path, 'apply_gimbal.py')
path_to_gimbal = join(base_path, 'og_apply_gimbal.py')
slurm_out_path = join(base_path, "slurm_outs/slurm-%j.out")

In [3]:
prefixes = []
for k in good_sessions:
    mouse = k.split('_')[-1]
    prefixes.append(f'/n/groups/datta/Jonah/kpms_reviews_6cam_thermistor/raw_data/{mouse}/{k}')


### center detect

In [4]:
slurm_cmd = f'sbatch -p gpu_quad -t 1:15:00 --mem 4GB --gres=gpu:1 -c 2 -o {slurm_out_path} --wrap '  # for 30 min vids on CW rig, 1 hr usually enough.
wrap_cmd = f'\'eval "$(conda shell.bash hook)"; conda activate dataPy_torch; module load ffmpeg; module load gcc/9.2.0; python {path_to_centernet} {{}}\''
cmd = slurm_cmd + wrap_cmd
overwrite = False
for prefix in prefixes:
    for vid_path in glob.glob(join(prefix, video_glob)):
        if exists(vid_path.replace(".avi", ".centroid.npy")) and not overwrite:
            continue
        print(cmd.format(vid_path))
        print()

### hrnet

In [5]:
# ~4 hrs for a 30 min vid on CW rig
# ~20% of jobs timed out at 4:30:00 around 90%...try 4.5*1.1 ~= 5:00:00.
slurm_cmd = f'sbatch -p gpu_quad -t 5:15:00 --mem 4GB --gres=gpu:1 -c 2 -o {slurm_out_path} --wrap '
wrap_cmd = f'\'eval "$(conda shell.bash hook)"; conda activate dataPy_torch; module load ffmpeg; module load gcc/9.2.0; python {path_to_hrnet} {{}}\''
cmd = slurm_cmd + wrap_cmd
overwrite = False
for prefix in prefixes:
    for vid_path in glob.glob(join(prefix, video_glob)):
        putative_kp_path = vid_path.replace(".avi", ".keypoints.h5")
        if exists(putative_kp_path) and not overwrite:
            try:
                # Check we can open the h5 file -- sometimes if a job gets cut off, we'll need to re-run it
                # (TODO: make h5 checkpoints so that if it fails one frame from the end, it doesn't have to be all re-run)
                with h5py.File(putative_kp_path,'r') as h5:
                    continue
            except OSError:
                pass
        print(cmd.format(vid_path))

#### Check for incomplete h5 files (will need to be re-run)

In [6]:
import h5py
bad_files = []
vid_suffix = '.avi'  # avi or mp4
for prefix in prefixes:
    for vid_path in glob.glob(join(prefix, video_glob)):
        comp_kp_path = vid_path.replace(vid_suffix, '.keypoints.h5')
        try:
            with h5py.File(comp_kp_path,'r') as h5:
                pass
        except OSError:
            bad_files.append(comp_kp_path.replace('.keypoints.h5', vid_suffix))

In [7]:
len(bad_files)

0

In [8]:
bad_files

[]

In [9]:
for prefix in prefixes:
    for vid_path in glob.glob(join(prefix, video_glob)):
        if vid_path not in bad_files: continue
        print(cmd.format(vid_path))

### triangulate

In [5]:
calib_mapping = {
    '20230904_J01601':'20230904_calibration',
    '20230905_J01601':'20230905_calibration',
    
    '20230904_J01602':'20230904_calibration',
    '20230905_J01602':'20230905_calibration',
    
    '20230904_J01701':'20230904_calibration',
    '20230905_J01701':'20230905_calibration',
    '20230911_J01701':'20230911_calibration',
    '20230912_J01701':'20230912_calibration',
    
    '20230905_J01702':'20230905_calibration',
    '20230911_J01702':'20230911_calibration',
    '20230912_J01702':'20230912_calibration',
    
    '20230904_J01703':'20230904_calibration',
    '20230908_J01703':'20230908_calibration',
    '20230911_J01703':'20230911_calibration',
    '20230912_J01703':'20230912_calibration',
    
}

In [6]:

# slurm_cmd = 'sbatch -p gpu_quad -t 1:00:00 --mem 6GB --gres=gpu:1 -c 2 --wrap '
slurm_cmd = f'sbatch -p short -t 1:00:00 --mem 6GB -c 1 -o {slurm_out_path} --wrap '
wrap_cmd = f'\'eval "$(conda shell.bash hook)"; conda activate dataPy_NWB2; module load ffmpeg; module load gcc/9.2.0; python {path_to_triangulate} {{}} {{}}\''
cmd = slurm_cmd + wrap_cmd

output_name = 'robust_triangulation.npy'
overwrite = False

for vid,calib in calib_mapping.items():
    
    calib_path = '/n/groups/datta/Jonah/kpms_reviews_6cam_thermistor/raw_data/calibration/data/'+calib+'/camera_params.h5'
    mouse = vid.split('_')[-1]
    vid_path = f'/n/groups/datta/Jonah/kpms_reviews_6cam_thermistor/raw_data/{mouse}/{vid}'
    if not exists(calib_path):
#         print(f'Could not find calib {calib_path}')
        continue    
    if os.path.exists(join(vid_path, output_name)) and not overwrite:
        continue
    print(cmd.format(vid_path, calib_path))
    print()

### gimbal

In [8]:
# about 5 hrs per 30 min video
slurm_cmd = f'sbatch -p gpu_quad -t 5:30:00 --mem 8GB --gres=gpu:1 -c 1 -o {slurm_out_path} --wrap  '
wrap_cmd = f'\'eval "$(conda shell.bash hook)"; conda activate dataPy_gimbal; module load ffmpeg; module load gcc/9.2.0; module load cuda/11.7; python {path_to_gimbal} {{}} {{}}\''
cmd = slurm_cmd + wrap_cmd
output_name = 'gimbal.npy'

for vid,calib in calib_mapping.items():
    calib_path = '/n/groups/datta/Jonah/kpms_reviews_6cam_thermistor/raw_data/calibration/data/'+calib+'/camera_params.h5'
    mouse = vid.split('_')[-1]
    vid_path = f'/n/groups/datta/Jonah/kpms_reviews_6cam_thermistor/raw_data/{mouse}/{vid}'
    if os.path.exists(join(vid_path, output_name)) and not overwrite:
        print("exists")
        continue
    print(cmd.format(calib_path, vid_path))
    print()

exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
exists
